In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
    !pip install -q transformers
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/MyDrive/gan_bert_for_cpv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.3 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
from transformers import logging
logging.set_verbosity_error()
import torch
from create_dataset import CreateDataset
from process_data import ProcessData
from utils import *
from torch.utils.data import DataLoader
from run_model import RunModel
from models import BertGenerator, BertDiscriminator
from transformers import AutoTokenizer, AutoModel, AutoConfig

In [3]:
from models import *

In [4]:
label_column = "afdeling"
text_column = "Korte beschrijving aanbesteding"
input_df = pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_dutch_ted1.csv")

In [5]:
vc = input_df[label_column].value_counts()
down_sample_vc = vc[vc.values < 1500]
input_df = input_df[input_df[label_column].isin(down_sample_vc.index)]

In [6]:
running_device = torch.device("cuda")

In [7]:
prd = ProcessData(input_df, label_column, text_column, sample_num=2000, aug_limit=1000,test_per_label_num=30, drop_limit=100,
                  balanced_test=True)
_,_,_, label_encoder, onehot = prd.process()

After removing duplicate sentence: 16897
After drop cpv with low observations we have : 16681
The num of class is  28
The number of total data is:  16681
Total number of test data is:  840
Total number of train data is:  15841


In [8]:
train_df = pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/sampled_train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/sampled_train.csv")
value_counts = train_df[label_column].value_counts()
model_runner = RunModel(running_device)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
config = AutoConfig.from_pretrained("GroNLP/bert-base-dutch-cased", output_hidden_states=True)
bert = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased", config=config).to(running_device)

In [ ]:
cd = CreateDataset(text_column, label_column, label_encoder, onehot, 11, running_device, max_length=100)
test_dataset = cd.create(test_df)
suf_dataset = cd.create(suf_df)
test_dl = DataLoader(test_dataset, shuffle=True, batch_size=20)
insuf_ds_dict = cd.create_label_dataset(insuf_df)
insuf_ds_list = list(insuf_ds_dict.values())
original_insuf_dataset = torch.utils.data.ConcatDataset(insuf_ds_list)
original_train_dataset = torch.utils.data.ConcatDataset([suf_dataset, original_insuf_dataset])
balanced_num = 1500
batch_size = 20
gan_aug_datasets = []
copy_aug_datasets=[]
for label in insuf_ds_dict:
    sub_ds = insuf_ds_dict[label]
    generator = BertGenerator(bert)
    discriminator = BertDiscriminator(bert)
    enrich_num = balanced_num - len(sub_ds)
    enrich_batch = int(enrich_num / batch_size)*2
    source_train = DataLoader(sub_ds, shuffle=True, batch_size=20)
    generator, discriminator = model_runner.generator_trainer(generator, discriminator,
                                  source_train, source_train,
                                  batch_num=20, n_epochs=10, 
                                  glr=2e-6, dlr=2e-6)
    one_data, one_label = sub_ds[0]
    size = (batch_size, one_data.size(0), one_data.size(1))
    gan_aug_dataset = model_runner.generate_aug_data(generator, discriminator, 
                              one_label, size=size,batch_num=enrich_batch)
    print(len(sub_ds))
    print(len(gan_aug_dataset))
    gan_aug_datasets.append(gan_aug_dataset)

In [ ]:
gan_aug_train_dataset = torch.utils.data.ConcatDataset(gan_aug_datasets + [original_train_dataset])
hybrid_aug_dataset=torch.utils.data.ConcatDataset(copy_aug_datasets + gan_aug_datasets)
gan_train_dl = DataLoader(gan_aug_train_dataset, shuffle=True, batch_size=20)

In [ ]:
classifier = HiddenClassifier11(prd.num_class, bert)
classifier = model_runner.train_classifier(gan_train_dl, test_dl, classifier, lr=1e-4, n_epochs=10)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recal=evaluate(labels, predictions)

100%|██████████| 2050/2050 [00:51<00:00, 39.64it/s]


0 train loss:  1.1394478016010507 val acc:  0.5190476321038746


100%|██████████| 2050/2050 [00:51<00:00, 39.94it/s]


1 train loss:  0.7748148375064016 val acc:  0.5476190582627342


100%|██████████| 2050/2050 [00:51<00:00, 39.85it/s]


...

100%|██████████| 2050/2050 [00:51<00:00, 39.87it/s]


8 train loss:  0.318343297060713 val acc:  0.5785714430468423


100%|██████████| 2050/2050 [00:51<00:00, 39.87it/s]


9 train loss:  0.3125378962342365 val acc:  0.57380953005382
Total ACC: 0.574
F1: 0.557
Precision: 0.590
Recall: 0.574


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gan enriched data

In [ ]:
copy_aug_insuf_dataset = []
for label in insuf_ds_dict:
    ds = insuf_ds_dict[label]
    aug_insuf_ds = [ds[i % len(ds)] for i in range(balanced_num)]
    copy_aug_insuf_dataset += aug_insuf_ds

NameError: ignored

In [ ]:
copy_aug_train_dataset = torch.utils.data.ConcatDataset(copy_aug_insuf_dataset + [original_train_dataset])

# Only copy enriched dataset

In [ ]:
copy_train_dl = DataLoader(copy_aug_train_dataset, shuffle=True, batch_size=20)
classifier = HiddenClassifier9(prd.num_class, bert).to(running_device)
classifier = model_runner.train_classifier(copy_train_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
evaluate(labels, predictions, prd.label_encoder, prd.num_class, value_counts)

# Hybrid enriched train dataset; copy+gan augmented

In [ ]:
hybrid_train_dataset = torch.utils.data.ConcatDataset(
    gan_aug_datasets + copy_aug_insuf_dataset + [original_train_dataset])
hybrid_train_dl = DataLoader(hybrid_train_dataset, shuffle=True, batch_size=20)
classifier = HiddenClassifier9(prd.num_class, bert).to(running_device)
classifier = model_runner.train_classifier(hybrid_train_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall = evaluate(labels, predictions, prd.label_encoder, prd.num_class, value_counts)

# Only original dataset training

In [ ]:
original_dl = DataLoader(original_train_dataset, shuffle=True, batch_size=20)
classifier = HiddenClassifier1(prd.num_class, bert).to(running_device)
classifier = model_runner.train_classifier(original_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall = evaluate(labels, predictions)

0 train loss:  2.9717725387587675 val acc:  0.3047619096580006
1 train loss:  1.9207754454576653 val acc:  0.39761905443100704
2 train loss:  1.440107233235131 val acc:  0.5261904895305634
3 train loss:  1.1035037373308876 val acc:  0.5464285817884263
4 train loss:  0.8879791532007772 val acc:  0.5714285834914162
5 train loss:  0.7403410421226357 val acc:  0.5845238211609068
Total ACC: 0.585
F1: 0.525
Precision: 0.491
Recall: 0.585


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
